In [ ]:
# Imports
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

# Environment setup
load_dotenv()
deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
language="python"

#Initialize AzureOpenAI client
client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],  
  api_version = "2023-12-01-preview"
  )

messages=[]

In [ ]:
def select_machine_family(machineKind):
    if machineKind == "cutting":
        return "cut"
    elif machineKind == "packing":
        return "pack"
    elif machineKind == "soldering":
        return "solder"
    else:
        return ""

def select_machine_operation(machine_family, direction):    
    if machine_family == "cut":
        if direction=="vertical":
            return "cut_vertical"
        elif direction=="horizontal":
            return "cut_horizontal"
        else:
            return "cut_round"
    elif machine_family == "solder":
        if direction=="vertical":
            return "s_vertical"
        elif direction=="horizontal":
            return "s_horizontal"
        else:
            return "s_round"        
    #TODO: add other machine families
    else:
        return ""
    
def select_external_library_for_cutting(machineModel, operation):    
    if machineModel == "SE-01":
        if operation=="cut_vertical":
            return  '''
                The library package "machine_cut_v" contains functions related to material cutting.
                
                Here's the list of available functions:
                         
                # Cut the material vertically from the point (beginX,beginY) to the point (endX,endY)
                verticalCut(beginX, beginY, endX, endY)
                
                # Cut the material vertically from the point (beginX,beginY) to the point (endX,endY) in a zigzag pattern
                verticalCutZZ(beginX, beginY, endX, endY)        
                '''
        elif operation=="cut_horizontal":
            return  '''
                The library package "machine_cut_h" contains functions related to material cutting.
                
                Here's the list of available functions:
                         
                # Cut the material horizontally from the point (beginX,beginY) to the point (endX,endY)
                HCut(beginX, beginY, endX, endY)
                
                # Cut the material horizontally from the point (beginX,beginY) to the point (endX,endY) in a zigzag pattern
                HZZCut(beginX, beginY, endX, endY)
                '''
        elif operation=="cut_round":
            return  '''
                The library package "machine_cut_r" contains functions related to material cutting.
                
                Here's the list of available functions:
                         
                # Creates a round cut where point (beginX,beginY) is the center and radius is the radius of the circle
                RoundCut(beginX, beginY, radius)
                
                # Creates a round cut where point (beginX,beginY) is the center, radius is the radius of the circle and border follows a zigzag pattern
                RoundCutZigZag(beginX, beginY, endX, endY)
                '''
        else:
            return ""
    #TODO: add other machine families
    else:
        return ""
    
def select_external_library_for_soldering(machineModel, operation):    
    if machineModel == "SE-01":
        if operation=="s_vertical":
            return  '''
                The library package "machine_solder_v" contains functions related to material soldering.
                
                Here's the list of available functions:
                         
                # Solder the material vertically from the point (beginX,beginY) to the point (endX,endY)
                vSold(beginX, beginY, endX, endY)
                
                # Solder the material vertically from the point (beginX,beginY) to the point (endX,endY) in a zigzag pattern
                v_S_ZZCut(beginX, beginY, endX, endY)        
                '''
        elif operation=="s_horizontal":
            return  '''
                The library package "machine_solder_h" contains functions related to material soldering.
                
                Here's the list of available functions:
                         
                # Weld the material horizontally from the point (beginX,beginY) to the point (endX,endY)
                sold_H(beginX, beginY, endX, endY)
                
                # Weld the material horizontally from the point (beginX,beginY) to the point (endX,endY) in a zigzag pattern
                H_ZZ_Sold(beginX, beginY, endX, endY)
                '''
        elif operation=="s_round":
            return  '''
                The library package "machine_weld_circular" contains functions related to material soldering.
                
                Here's the list of available functions:
                         
                # Performs a circular welding of the material where (beginX,beginY) is the center and radius is the radius of the circle
                RoundWeld(beginX, beginY, radius)
                
                # Performs a circular welding of the material where (beginX,beginY) is the center, radius is the radius of the circle and border follows a zigzag pattern
                Round_W_ZigZag(beginX, beginY, endX, endY)
                '''
        else:
            return ""
    #TODO: add other machine families
    else:
        return ""



# Load function in a dictionary for later use
available_functions = {
                "select_machine_family": select_machine_family,
                "select_machine_operation": select_machine_operation,
                "select_external_library_for_cutting": select_external_library_for_cutting,
                "select_external_library_for_soldering": select_external_library_for_soldering,
        }

In [ ]:
functions = [
   {
      "name":"select_machine_family",
      "description":"Select the family of machine to use for library selection",
      "parameters": {
         "type":"object",
         "properties":{
            "machineKind":{
               "type":"string",               
               "description": "The family of the machine, can be one of the following: cutting, packing, soldering."
            },         
         },
         "required":["machineKind"]
      },
   },  
   {
      "name":"select_machine_operation",
      "description":"Select the family of operation to use for library selection",
      "parameters": {
         "type":"object",
         "properties":{
            "machine_family":{
               "type":"string",               
               "description": "The family of the machine, can be one of the following: cut, pack, solder."
            },
            "direction":{
               "type":"string",               
               "description": "The kind of cut operation, can be one of the following: vertical, horizontal, round."
            },          
         },
         "required":["machineKind","operation"]
      },
   },  
   {
      "name":"select_external_library_for_cutting",
      "description":"Select the external libraries that can be used for in case of a cutting operation.",
      "parameters": {
         "type":"object",
         "properties":{
            "machineModel":{
               "type":"string",               
               "description": "The model of the machine the cutting operation will be executed."
            },
            "operation":{
               "type":"string",               
               "description": "The direction of the cut operation, can be one of the following: cut_vertical, cut_horizontal, cut_round."
            },          
         },
         "required":["machineModel","direction"]
      },      
   },  
   {
      "name":"select_external_library_for_soldering",
      "description":"Select the external libraries that can be used for in case of a welding operation.",
      "parameters": {
         "type":"object",
         "properties":{
            "machineModel":{
               "type":"string",               
               "description": "The model of the machine the soldering operation will be executed."
            },
            "operation":{
               "type":"string",               
               "description": "The direction of the soldering operation, can be one of the following: s_vertical, s_horizontal, s_round."
            },          
         },
         "required":["machineModel","direction"]
      },      
   },  
]

In [ ]:
# system message
system_message = f'''
You are a an expert in generating code using exclusively the {language} language.
Based on the user request you will generate the code for the user following the classic {language} conventions and commenting the code so that it will be easily readable favoring semplicity over complexity.
The generated code should be syntactically correct and should be able to run without errors and the it should include the following features:
- All the required imports should be included and listed at the top of the file.
- The code should be executed via a main function.
- Do not generate additional ouput comments other than the inline with the code.
- Do not generate additional code other than the one requested by the user (example code to initialize the machine, etc.).

In order to generate compliant code you might need to use external libraries, use function provided in the prompt to request detailed information about them.

'''

In [ ]:
function_extraction_rules = f'''
The following rules should be used to identify the functions to use from the external libraries:
-Use a specific external function only if:
    - The function name seems to be designed for the specific use case (e.g. blow_air() in case the code should blow air)
    - The optional function comments or related description seems to match the functionality.
    - Associated function comments or related documentation seems to describe the required functionality (e.g. turn_on_machine() is associated with a comment or documentaion that says "This function turns on the machine") despite the name of the function itself seems to disagree:
        Examples:
        - the name of the function is: turn_on_system() but the comment says "turns on the machine" in this case the function should be used when the code should turn on the machine.
        - the name of the function is: HSI(sec) and the comment says "It heats the welding head for the specified amount of seconds" in this case the function should be used when the code is supposed to heat a soldering head.
    - Favor the description of the function over the name of the function, to decide if the function should be used or not.
    - The documentation indicates that the function can be used for the required functionality:
        Examples of when it should NOT be use:
        - Documentation says "This function can be used to turn on the machine (deprecated)" in this case the function should be used because deprecated.
        - Documentation indicates other better alternatives like "This function can be used to turn on the machine (note: use turn_on_machine2() instead)" in this case the suggested function turn_on_machine2() should be picked.

-Do not use the external library functions if:
    - the name of the function seems to be designed for other specific use cases (e.g. turn_on_machine() seems to not be designed to blow air)    
if it appears to have external functions that overlaps in functionality (e.g. blow_air_for(seconds) and blow_air_on() and blow_air_off()) favor the one that result in lesser generated code for the requested functionality.
if none of the external libraries offer a function for the required functionality, insert a comment similar to the following one:

    # NOTE: no external function found to <replace with name the operation to achieve (e.g. 'blow air')>
    <insert a comment with a pseudo library function call, possibly using a meaningful name (e.g. turn_on_machine in case required functionality is to turn on the machine instead of foo() or xxx()) including required parameters that can be used to achieve the required functionality>
    
    the comment shold be placed before the pseudo library function call and should be indented with the same indentation of the pseudo library function call.
    the pseudo library function call should be commented out.

'''

Theses are the user commands, edit and run them to see the different output considering that output also depends on code implemented into function (to be replaced by RAG)

In [ ]:

# Prompt #1 cut

# user_command = f'''
# Generate a program to control a machine model SE-01 via a programmable logic controller (PLC) to cut a material, in a way that reminds a circle following a zigzag pattern from the point (10,10) to the point (100,100).
# '''

# Prompt #2 soldering
# user_command = f'''
# Generate a program to control a machine model SE-01 to solder a material vertically following a zigzag pattern from the point (10,10) to the point (100,100).
# '''

# Prompt #3 cutting and soldering
user_command = f'''
Generate a program to control a machine model SE-01 via a programmable logic controller (PLC) to execute the following operations:

1. cut the material, vertically from the point (10,10) to the point (100,100).
2. create a round cut where point (123,456) is the center and 78 is the radius of the circle.
3. solder the material, horizontally from the point (65,18) to the point (178,481).
4. create a circular welding of the material from center (742,95) with a radius of 12 and border follows a zigzag pattern.

'''

In [ ]:
# Load info required by the model    
def load_info(response_message):
    function_name = response_message.function_call.name
    if function_name:
        print(f"\nModel wants to call a function '{function_name}'...") 
        function_to_call = available_functions[function_name] 

        function_args = json.loads(response_message.function_call.arguments)
        function_response = function_to_call(**function_args)
        print(f"Function '{function_name}' returned: {function_response}")
        # Append function info to messages
        messages.append( # adding assistant response to messages
            {
                "role": response_message.role,
                "function_call": {
                    "name": function_name,
                    "arguments": response_message.function_call.arguments,
                },
                "content": None
            }
        )
        messages.append( # adding function response to messages
            {
                "role": "function",
                "name": function_name,
                "content":function_response,
            }
       ) 

In [ ]:
# Asks the model to generate the code
def generate_code():
    openai_response = client.chat.completions.create(
        model=deployment,    
        messages = messages,
        temperature=0.3,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        functions=functions,
        function_call="auto"
    )
    return openai_response.choices[0].message

In [ ]:
messages.append({'role': 'system', 'content': system_message})
messages.append({'role': 'user', 'content': function_extraction_rules})
messages.append({'role': 'user', 'content': user_command})       

isCodeGenerated=False

while(isCodeGenerated==False): 
    print("Generating code...")    
    response = generate_code()    
    if (response.function_call):
        load_info(response)
    else:
        isCodeGenerated=True
        print("Code generation completed!")        
        print(response.content)